%run ./mlutils/tfutil.ipynb import flags, stop_here

In [2]:
#!/usr/bin/env python3

#
# pip3 install tf-nightly-gpu 
# pip3 install tensorflow-datasets
# conda install -n tf2gpu scikit-learn  scikit-image matplotlib pandas tqdm opencv  matplotlib
# pip3 install -U tensorboard 
# On windows, path lengths greater than 260 characters may result in an error.
#  - https://docs.python.org/3/using/windows.html#removing-the-max-path-limitation 

_= r''' [code snippet]
import datetime
tick = f'{datetime.datetime.now():%Y%m%d-%H%M%S}'


import matplotlib.pyplot as plt
%matplotlib inline


'''

In [3]:

import os
import sys
import io

# import sys, os
# sys.path.append(r'D:\github\mydir')
# from jjkim_util import *

# color-text output
black, red, green, yellow, blue, magenta, sky, white = range(30, 38)


def clrprint(fg, *args, **kwargs) -> None:
    '''
    fg: text color
    args: args for print()
    kwargs: bg=color_number + kwargs for print()
    '''
    if 'bg' in kwargs:
        print(f"\x1b[{fg};{kwargs['bg']+10}m", end='')
        del kwargs['bg']
    else:
        print(f"\x1b[{fg}m", end='')
    print(*args, end='\x1b[0m')
    print('', **kwargs)


def clrprint_dump():
    for fg in range(30, 38):
        for bg in range(30, 38):
            if fg == bg:
                continue
            clrprint(fg, f'clrprint({fg},{bg}, )', end='  ', bg=bg)
        print()


def clrcode_dump():
    for fg in range(30, 38):
        for bg in range(30, 38):
            if fg == bg:
                continue
            clrprint(
                fg, rf' \x1b[{fg};{bg+10}m TEXT_HERE \x1b[0m ', bg=bg, end='  ')
        print()


warn = (lambda *args, **kwargs: clrprint(yellow, *args, **kwargs))
err = (lambda *args, **kwargs: clrprint(red, *args, **kwargs))
info = (lambda *args, **kwargs: clrprint(green, *args, **kwargs))
debug = print



In [4]:
# python utils
def static_vars(**kwargs):
    '''
     @static_vars(label_info= ds_info.features["label"])
     '''
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

# @static_vars(label_info= ds_info.features["label"])


In [5]:

#
# juypter helper
#


def stop_here(msg='Early stopped!'):
    '''
    stop execute a cell without reset kernel.
    '''
    import inspect

    try:
        class EarlyStop(InterruptedError):
            pass

        def except_handler(shell, etype, evalue, tb, tb_offset=None):
            sys.stderr.close()
            sys.stderr = sys.__stderr__

        get_ipython().set_custom_exc((EarlyStop,), except_handler)
        sys.stderr = io.StringIO()
    except NameError as ex:
        err(ex)
        return

    lineno = inspect.getframeinfo(inspect.currentframe().f_back).lineno
    print(f'\x1b[33;40m\n      {msg} at line {lineno}     \n\x1b[0m')
    raise EarlyStop


def print_codes_only_be_used_in_ipynb():
    print('''
    
nb_folder = globals()['_dh'][0] # get this local notebook(xxx.ipynb) folder
    ''')


# tf related

In [6]:
import logging
import tensorflow as tf

def tf_set_loglevel(lvl):
    '''
    logging.WARNING
    logging.ERROR
    '''
    import tensorflow
    tensorflow.get_logger().setLevel(lvl)

    
def disable_tfgpu():
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # or even "-1"


def dump_gpu():
    print('tf version:', tf.__version__)
    # check Cuda & GPU
    print('is_built_with_cuda:', tf.test.is_built_with_cuda())
    print('is_built_with_gpu_support:', tf.test.is_built_with_gpu_support())

    from tensorflow.python.client import device_lib
    local_devices = device_lib.list_local_devices()
    for x in local_devices:
        print('>', x.device_type)
        print(
            f'   memory_limit: {x.memory_limit/(1024.0*1024.*1024.):.2f}GB, {x.memory_limit}B')
        if (x.physical_device_desc):
            name = x.physical_device_desc
            print(f'   {name}')
        #print('x:', x)
    return


def tf_growable_gmem():
    import tensorflow as tf
    gpus = tf.config.list_physical_devices('GPU')
    for i in gpus:
        tf.config.experimental.set_memory_growth(i, True)
        print('set_memory_growth true:', i)

def tfconfig(grow_gmem=True):
    import tensorflow as tf
    print('tf version:', tf.__version__)
    # check Cuda & GPU
    print('is_built_with_cuda:', tf.test.is_built_with_cuda())
    print('is_built_with_gpu_support:', tf.test.is_built_with_gpu_support())

    from tensorflow.python.client import device_lib
    local_devices = device_lib.list_local_devices()
    for x in local_devices:
        print('>', x.device_type)
        print(
            f'   memory_limit: {x.memory_limit/(1024.0*1024.*1024.):.2f}GB, {x.memory_limit}B')
        if (x.physical_device_desc):
            name = x.physical_device_desc
            print(f'   {name}')

    if grow_gmem:
        gpus = tf.config.list_physical_devices('GPU')
        for i in gpus:
            tf.config.experimental.set_memory_growth(i, True)
            print('set_memory_growth true:', i)
    return

def check_cuda():
    import subprocess
    import glob
    # stdout=subprocess.DEVNULL)
    ret = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    print(ret.stdout)

    cudnn_folders = [
        r'C:/Program Files/NVIDIA/CUDNN',
        r'C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA',
    ]
    for folder in cudnn_folders:
        if not os.path.exists(folder):
            continue
        files = glob.glob(os.path.join(
            folder, '**/cudnn_version.h'), recursive=True)
        if len(files) > 0:
            os.startfile(files[0])

#
#
#
class flags:
    '''
     flags for notebook. alternative for absl.flags
    '''
    class AttrDict(dict):
        def __init__(self):
            self.__dict__ = self
        def __repr__(self) -> str:
            lines = []
            for k,v in flags.FLAGS.items():
                lines.append(f'{k:30s} = {v}')
            return '\n'.join(lines)

    FLAGS = AttrDict()

    @staticmethod
    def DEFINE_string(key, default, help = None):
        flags.FLAGS[key] = str(default)

    @staticmethod
    def DEFINE_integer(key, default, help = None):
        flags.FLAGS[key] = int(default)

    @staticmethod
    def DEFINE_float(key, default, help = None):
        flags.FLAGS[key] = float(default)

    @staticmethod
    def DEFINE_bool(key, default, help = None):
        flags.FLAGS[key] = bool(default)
    @staticmethod
    def DEFINE_object(key, default, help = None):
        flags.FLAGS[key] = default


ModuleNotFoundError: No module named 'tensorflow.python.compiler.tensorrt.trt_convert_windows'

In [ ]:
# tf checkpoint
import tensorflow as tf

class TFCheckPoint:
  def __init__(self, model, path, step):
    self.model = model
    self.ckpt = tf.train.Checkpoint(step=tf.Variable(0), model=model)
    self.ckpt_manager = tf.train.CheckpointManager(self.ckpt, directory=path, max_to_keep=None)
    self.save_step = step
    if latest_ckpt := tf.train.latest_checkpoint(path):
      self.ckpt.restore(latest_ckpt)
      print('global_step : {}, checkpoint is restored!'.format(int(self.ckpt.step)))

  def inc_step(self):
    self.ckpt.step.assign_add(1) # global step

  def save(self):
    if (self.ckpt.step % self.save_step) == 0:
      self.ckpt_manager.save(checkpoint_number=self.ckpt.step)
      print('global_step : {}, checkpoint is saved!'.format(int(self.ckpt.step)))
    # self.inc_step()


# Filesystem


In [ ]:
import glob
import os

def get_latest_file(folder_path, file_type=''):
    '''
    get latest .ext file
    '''
    if not os.path.isdir(folder_path):
        raise FileNotFoundError(folder_path)

    files = glob.glob(os.path.join(folder_path, '*' + file_type))
    if len(files) == 0:
        raise FileNotFoundError(os.path.join(folder_path, '*' + file_type))

    return max(files, key=os.path.getctime)

In [ ]:

#
#
#
def helper():
    clrprint_dump()
    clrcode_dump()

    warn("warn")
    err("err")
    info("info")
    debug('debug')

#
#
#
if __name__ == '__main__':
    info('tfutil.ipynb loaded')
    pass
